##### GA parameters

In [574]:
# GA parameter sets: baseline, exploration and exploitation.
GA_PARAMETER_SETS = [
    {
        "name": "Baseline set",
        "generation_number": 4,
        "population_size": 5,
        "crossover_probability": 0.8,
        "mutation_probability": 0.1
    },
    {
        "name": "Exploration scenario",
        "generation_number": 100,
        "population_size": 10,
        "crossover_probability": 0.9,
        "mutation_probability": 0.2
    },
    {
        "name": "Exploitation scenario",
        "generation_number": 30,
        "population_size": 8,
        "crossover_probability": 0.7,
        "mutation_probability": 0.001
    }
]


# dictionary for each scenario with instance 1 and two for all scenarios (small, medium and large)
PROBLEM_INSTANCES = [
    {"name": "Small instance 1", "num_customers": 14, "num_vehicles": 3},
    {"name": "Small instance 2", "num_customers": 20, "num_vehicles": 10},
    {"name": "Medium instance 1", "num_customers": 21, "num_vehicles": 10},
    {"name": "Medium instance 2", "num_customers": 28, "num_vehicles": 20},
    {"name": "Large instance 1", "num_customers": 32, "num_vehicles": 20},
    {"name": "Large instance 2", "num_customers": 50, "num_vehicles": 25}
]

print("Genetic Algorithm parameter sets:")
for GA_set in GA_PARAMETER_SETS:
    print(f"{GA_set['name']} has {GA_set['generation_number']} number of generations, the crossover probability is {GA_set['crossover_probability']} and mutation probability is {GA_set['mutation_probability']}")
print("-----------------------------------------------------------------")
print("Problem instances:")
for instance in PROBLEM_INSTANCES:
    print(f"{instance['name']} has {instance['num_customers']} customers and {instance['num_vehicles']} vehicles.")


Genetic Algorithm parameter sets:
Baseline set has 4 number of generations, the crossover probability is 0.8 and mutation probability is 0.1
Exploration scenario has 100 number of generations, the crossover probability is 0.9 and mutation probability is 0.2
Exploitation scenario has 30 number of generations, the crossover probability is 0.7 and mutation probability is 0.001
-----------------------------------------------------------------
Problem instances:
Small instance 1 has 14 customers and 3 vehicles.
Small instance 2 has 20 customers and 10 vehicles.
Medium instance 1 has 21 customers and 10 vehicles.
Medium instance 2 has 28 customers and 20 vehicles.
Large instance 1 has 32 customers and 20 vehicles.
Large instance 2 has 50 customers and 25 vehicles.


##### Create customers and locations data frame

In [573]:
import pandas as pd

all_customers_df = pd.read_csv("customers.csv")
print("Customer CSV successfully loaded.")
print("Number of customers: ", len(all_customers_df))


def combine_coordinates(customers_df):
    """
    Add depot coordinates to customers list.

    :param customers_df:
    :return:
    """
    all_coordinates_df = pd.DataFrame()
    ######## 2: Add depot location to data frame ########
    try:
        all_coordinates_df = pd.concat(
        [customers_df,
         pd.DataFrame([[0] + [50,50]], columns=customers_df.columns)])
        print("Depot location successfully added to all_coordinates_df.")
    except ValueError:
        print("Cannot extract coordinates from customer CSV.")

    # Returns df with all customer and with all coordinates (customers + depot loc)
    return all_coordinates_df



Customer CSV successfully loaded.
Number of customers:  50


##### Functions for genetic algorithm

In [578]:
import numpy as np
import random

# create individual chromosome
def create_individual(possible_customers_df, num_selected_customers, num_vehicles):
    """
    Creates the first random individual chromosome by shuffling customer ids.
    :param possible_customers_df: Data frame of all customers
    :param num_selected_customers: Number of selected customers for individual scenario
    :param num_vehicles: Number of vehicles for individual scenario
    :return: individual chromosome
    """

    all_customer_ids = possible_customers_df['id'].tolist()

    chromosome = random.sample(all_customer_ids, num_selected_customers)
    random.shuffle(chromosome)


    chromosome = add_vehicles(chromosome)

    return chromosome


def add_vehicles(individual):
        """
        Adds number of vehicles to individual chromosome by adding trips to depot.

        :param individual: individual chromosome with only customers.
        :return: new individual chromosome with customers and vehicles as list.
        """

        num_internal_depot_markers = num_vehicles - 1

        # place inner depot markers
        positions = list(range(1, len(individual)))         # possible placements for inner depot markers (not start, end or adjacent)
        individual_with_vehicles = individual.copy()        # new chromosome, ready to add depot markers into

        # shuffle possible positions of inner depot markers
        random.shuffle(positions)
        zeros_indices = []
        for pos in positions:
            if all(abs(pos - z) > 1 for z in zeros_indices):
                zeros_indices.append(pos)
                if len(zeros_indices) == num_internal_depot_markers:
                    break

        for idx in sorted(zeros_indices, reverse=True):
            individual_with_vehicles.insert(idx, 0)

        # create chromosome which start at 0, and end at 0, with the customers and random point of depot-visits inbetween
        individual_with_vehicles = [0] + individual_with_vehicles + [0]

        return individual_with_vehicles


######## 2. Create FITNESS function ########
def calculate_euclidean_distance(id1, id2, loc_df):
    """
    Calculates the Euclidean distance between two points.
    Based on formula: sqrt((x1 - x2)^2 + (y1 - y2)^2)

    :param id1: the id of first location
    :param id2: the id of second location
    :param loc_df: the dataframe containing the coordinates to all locations
    """

    # find x and y coordinated for id1
    x1 = loc_df.loc[loc_df['id'] == id1]['x'].iloc[0]
    y1 = loc_df.loc[loc_df['id'] == id1]['y'].iloc[0]

    # find x and y coordinates for id2
    x2 = loc_df.loc[loc_df['id'] == id2]['x'].iloc[0]
    y2 = loc_df.loc[loc_df['id'] == id2]['y'].iloc[0]

    # Euclidean calculation on objects id1 and id2
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def calculate_fitness(individual):
    """
    Calculate the fitness of the individual chromosome.

    :param individual:
    :return: the fitness of the individual chromosome (value between 0 and 1)
    """
    routes_segmented = []
    routes_distances = []
    start_index_of_route = 0

    try:
        # separate all vehicle trips into individual routes (list)
        # iterate through chromosome to find depot-markers and segment into individual vehicle routes.
        for i in range(len(individual)):

            if individual[i] == 0 and i != 0:                               # avoid counting starting depot point
                route_segment = individual[start_index_of_route: i + 1]     # create a temporary route segment for current vehicle trip
                routes_segmented.append(route_segment)                      # append list of all routes with new route segment

                # start to look for new route from where the last ended at the depot
                start_index_of_route = i

        # calculate distances for each trip
        for route in routes_segmented:

            # temporary list of individual Euclidean distances in one route segment
            single_route_distance = []

            for loc1,loc2 in zip(route, route[1:]):
                euclidean_distance = calculate_euclidean_distance(loc1, loc2, coordinates_df)
                single_route_distance.append(euclidean_distance)

            total_route_distance = sum(single_route_distance)
            routes_distances.append(total_route_distance)


        # sum distance of all trips
        total_accumulated_distance = sum(routes_distances)

        # calculate fitness based on total_trip_distance and return value
        return 1 / total_accumulated_distance

    except ZeroDivisionError:
        print("There are no routes to segment. Check if depot markers are present.")
        print("The route segmented is: ", individual)
        print("Number of depot markers present: ", individual.count(0))



######## 3. Create CROSSOVER function ########
def crossover_pmx(parent1, parent2, crossover_rate):
    """
    Calculates the crossover probability between two parents.

    :param crossover_rate: the rate of which crossover should take place.
    :param parent1: individual chromosome as list.
    :param parent2: individual chromosome as list.

    :return: child1: first child of parent - as chromosome list
    :return: child2: second child of parent - as chromosome list
    """

    # length of chromosome without depot markers. Flattened parent.
    length = len([val for val in parent1 if val != 0])
    inner_depot_markers = num_vehicles - 1
    child1 = [-1] * length
    child2 = [-1] * length

    if random.random() < crossover_rate:           # runs crossover if the randomly selected float is lower than the crossover rate

        # define crossover start and end
        start, end = sorted(random.sample(range(length), 2))

        child1[start:end] = parent1[start:end]
        child2[start:end] = parent2[start:end]

        # mapping between the parents and children. Maps duplicates
        mapping1_to_2 = {} # maps elements from segments of parent1 to parent2
        mapping2_to_1 = {} # Maps elements from segments of parent2 to parent1

        def resolve_mapping(gene, mapping, segment):
            """
            Follow the PMX mapping to resolve duplicates.
            """
            original_gene = gene
            visited = __builtins__.set()  # avoid shadowed built-ins

            while gene in segment:
                if gene in visited:
                    raise ValueError(f"Infinite mapping loop detected for gene {original_gene}")
                visited.add(gene)
                # Only follow mapping if gene is in mapping
                if gene in mapping:
                    gene = mapping[gene]
                else:
                    break  # gene not in mapping; safe to place

            return gene

        # fill remaining position in child 1
        for i in range(length):
            if start <= i < end:
                continue # Segment already copied

            child1[i] = resolve_mapping(parent2[i], mapping1_to_2, child1[start:end])

        # reintroduce vehicles to child 1
        child1 = add_vehicles(child1)

        for i in range(length):
            if start <= i < end:
                continue
            child2[i] = resolve_mapping(parent1[i], mapping2_to_1, child2[start:end])

        # reintroduce vehicles to child 2
        child2 = add_vehicles(child2)

    else:
        # no crossover if over crossover rate - children are copies of parents.
        child1 = parent1[:]
        child2 = parent2[:]

    return child1, child2



######## 4. Create MUTATION function ########
def mutate(individual, mutation_rate):
    """
    Mutates an individual chromosome.

    :param mutation_rate:
    :param individual: individual chromosome (list of customers)
    :return: the mutated chromosome (list of customers with slight mutations)
    """

    if random.random() < mutation_rate:           # randomly select a number and runs code if it is less than mutation rate

        # mutation procedure
        pos1, pos2 = random.sample(range(len(individual)), 2)                       # selects two distinct random positions in list
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]     # swap elements in positions

    return individual


_##### Genetic algorithm

In [570]:
# selection function
def tournament_selection(population, fitness, k=3):
    """
    Select one individual from population using tournament selection.
    """
    selected = random.sample(list(zip(population, fitness)), k)
    selected.sort(key=lambda x: x[1], reverse=True)  # higher fitness better
    return selected[0][0]  # return individual


# The genetic algorithm
def genetic_algorithm(population_size, num_selected_customers, num_vehicles, generations, crossover_rate, mutation_rate):
    """
    The genetic algorithm for the Vehicle Routing Problem.
    """

    # 1 - Create initial population
    population = [create_individual(all_customers_df, num_selected_customers, num_vehicles) for _ in range(population_size)]


    # 2 - Evaluate initial fitness
    fitness = [calculate_fitness(ind) for ind in population]

    best_individual = population[fitness.index(max(fitness))]
    best_fitness = max(fitness)
    print("Population size: ", population_size)
    print("best individual: ", best_individual)
    print("best fitness: ", best_fitness)


    # 3 - genetic algorithm applied through generations
    for gen in range(generations):
        new_population = []
        print("Generation: ", gen)

        # generate new population
        while len(new_population) < population_size:
            # tournament selection
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)

            # crossover
            child1, child2 = crossover_pmx(parent1, parent2, crossover_rate)

            # mutation
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        # trim excess if any
        population = new_population[:population_size]

        # evaluate fitness of new population
        fitness = [calculate_fitness(ind) for ind in population]

        # keep track of best solution
        current_best = population[fitness.index(max(fitness))]
        current_best_fitness = max(fitness)
        if current_best_fitness > best_fitness:
            best_individual = current_best
            best_fitness = current_best_fitness
        print("current best individual: ", best_individual)
        print("current best fitness: ", current_best_fitness)

        print(f"Generation {gen+1}: Best Fitness = {best_fitness}")


genetic_algorithm(4, 7, 2, 20, 0.8, 0.1)

Population size:  4
best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
best fitness:  0.0027645813813965505
Generation:  0
current best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
current best fitness:  0.0027645813813965505
Generation 1: Best Fitness = 0.0027645813813965505
Generation:  1
current best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
current best fitness:  0.0027645813813965505
Generation 2: Best Fitness = 0.0027645813813965505
Generation:  2
current best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
current best fitness:  0.0027645813813965505
Generation 3: Best Fitness = 0.0027645813813965505
Generation:  3
current best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
current best fitness:  0.0027645813813965505
Generation 4: Best Fitness = 0.0027645813813965505
Generation:  4
current best individual:  [0, 35, 23, 45, 31, 11, 48, 0, 28, 0]
current best fitness:  0.0027645813813965505
Generation 5: Best Fitness = 0.0027645813813965505
Generation: 

##### Analysis and results

In [ ]:
import matplotlib.pyplot as plt

def plot_customers(chromosome, customer_coords):
    """
    Plots customer locations on a 2D plane.

    :param chromosome:
    :param customer_coords: List of tuples [(x1, y1), (x2, y2), ...]
    """
    x = [c[0] for c in customer_coords]
    y = [c[1] for c in customer_coords]

    plt.scatter(x, y, c='blue', s=50, label='Customers')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Customer Locations')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_customers([44, 16, 42, 0, 38, 0, 0, 26, 30, 46] , all_customers_df)

In [572]:
original_list = [1, 0, 2, 0, 3, 4, 0]
print("Original list: ", original_list)

# Store indexes of zeros
zero_indexes = [i for i, val in enumerate(original_list) if val == 0]

# Remove zeros
no_zeros_list = [val for val in original_list if val != 0]

print("List without zeros:", no_zeros_list)
print("Indexes of zeros:", zero_indexes)

Original list:  [1, 0, 2, 0, 3, 4, 0]
List without zeros: [1, 2, 3, 4]
Indexes of zeros: [1, 3, 6]
